## Funcionalidades do Código

1. **Carregamento do Modelo**:
   - Carrega a estrutura do modelo CNN a partir de um arquivo JSON (`emotion_model.json`).
   - Carrega os pesos treinados do modelo a partir de um arquivo HDF5 (`emotion_model.h5`).

2. **Pré-processamento de Imagens**:
   - Redimensiona as imagens para 48x48 pixels.
   - Normaliza os valores dos pixels para a faixa [0, 1].
   - Converte as imagens para tons de cinza, adequando ao modelo treinado.

3. **Classificação de Emoções**:
   - Prediz a emoção presente em cada imagem usando o modelo treinado.
   - Usa um dicionário para mapear índices das classes para emoções humanas.

4. **Avaliação do Modelo**:
   - **Matriz de Confusão**:
     - Gera e exibe a matriz de confusão entre as classes verdadeiras e as preditas.
     - Visualiza a matriz graficamente.
   - **Relatório de Classificação**:
     - Exibe métricas de precisão, recall e F1-score para cada classe.

### Aplicações:
- Avaliação da precisão de modelos de detecção de emoções.
- Desenvolvimento de ferramentas de análise de emoções para pesquisas ou aplicações práticas.
- Identificação de melhorias para modelos de aprendizado profundo.

In [ ]:
# Importa as bibliotecas necessárias
import numpy as np
from keras.models import model_from_json  # Para carregar o modelo de um arquivo JSON
import matplotlib.pyplot as plt  # Para visualizações gráficas
from keras.preprocessing.image import ImageDataGenerator  # Para pré-processar imagens
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay  # Para avaliação de desempenho

# Dicionário que mapeia índices das classes para emoções humanas
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# --- Carregando o modelo ---
# Abre o arquivo JSON que contém a estrutura do modelo
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()  # Lê o conteúdo do arquivo
json_file.close()

# Reconstrói o modelo a partir do JSON carregado
emotion_model = model_from_json(loaded_model_json)

# Carrega os pesos treinados para o modelo
emotion_model.load_weights("model/emotion_model.h5")
print("Modelo carregado com sucesso do disco")

# --- Pré-processamento das imagens ---
# Cria um gerador de dados que aplica o reescalonamento de valores dos pixels para a faixa [0, 1]
test_data_gen = ImageDataGenerator(rescale=1./255)

# Prepara o gerador para o conjunto de dados de teste
test_generator = test_data_gen.flow_from_directory(
    'data/test',                # Diretório onde as imagens de teste estão armazenadas
    target_size=(48, 48),       # Redimensiona as imagens para 48x48 pixels
    batch_size=64,              # Define o tamanho do lote
    color_mode="grayscale",     # Converte as imagens para tons de cinza
    class_mode='categorical'    # Usa a classificação categórica
)

# --- Realizando predições ---
# Realiza predições sobre as imagens no conjunto de teste
predictions = emotion_model.predict_generator(test_generator)

# Comentário opcional: exibe as predições para cada imagem no conjunto de teste
# for result in predictions:
#     max_index = int(np.argmax(result))  # Identifica o índice com maior probabilidade
#     print(emotion_dict[max_index])      # Exibe a emoção correspondente

# --- Avaliação do modelo ---
print("-----------------------------------------------------------------")

# Gera a matriz de confusão comparando as classes verdadeiras com as predições
c_matrix = confusion_matrix(test_generator.classes, predictions.argmax(axis=1))
print("Matriz de Confusão:")
print(c_matrix)

# Exibe graficamente a matriz de confusão
cm_display = ConfusionMatrixDisplay(confusion_matrix=c_matrix, display_labels=list(emotion_dict.values()))
cm_display.plot(cmap=plt.cm.Blues)  # Define um esquema de cores azuis para o gráfico
plt.title("Matriz de Confusão - Detecção de Emoções")  # Adiciona um título ao gráfico
plt.show()

# Gera e exibe um relatório de classificação com métricas como precisão, recall e F1-score
print("Relatório de Classificação:")
print(classification_report(test_generator.classes, predictions.argmax(axis=1)))